# Introduction to `linopy`

:::{note}
This material is in part adapted from the following resources:
- [Linopy Tutorial](https://linopy.readthedocs.io/en/latest/index.html)
- [PyPSA simple electricity market examples](https://pypsa.readthedocs.io/en/latest/examples/simple-electricity-market-examples.html)
:::



[Linopy](https://linopy.readthedocs.io/en/latest/index.html)  is an open-source framework for formulating, solving, and analyzing optimization problems with Python.

With Linopy, you can create an optimization model within Python that consists of decision variables, constraints, and an optimization objective. You can then solve these instances using a variety of commerci and open-source solvers (specialised software).

[Linopy](https://linopy.readthedocs.io/en/latest/index.html) supports a wide range of problem types, including:

- **Linear programming**
- Quadratic programmingng
- Nonlinear programming
- Mixed-integer linear programming
- Mixed-integer quadratic programming
- Mixed-integer nonlinear programming
- Stochastic programming
- Generalized disjunctive programming
- Differential algebraic equations
- Bilevel programmin- Mathematical programs with equilibrium constraints


:::{note}
Documentation for this package is available at https://linopy.readthedocs.io/en/latest/index.html.
:::.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas pyomo highspy
```
:::

### Initializing a `Model`

The Model class in Linopy is a fundamental part of the library. It serves as a container for all the relevant data associated with a linear optimization problem. This includes variables, constraints, and the objective function.

In [1]:
from linopy import Model#, Variable, Objective#, Constraint, Sense

In [2]:
# Create a linopy Problem
m = Model()

This creates a new Model object, which you can then use to define your optimization problem.



### Adding variables

Variables in a linear optimization problem represent the decision variables. A variable can always be assigned with a lower and an upper bound. In our case, both `x` and `y` have a lower bound of zero (default is unbouded). In Linopy, you can add variables to a Model using the `add_variables` method:

In [3]:
# Create Variables

x = m.add_variables(lower=0, name='x')
y = m.add_variables(lower=0, name='y');
#m.x = Variable(domain=(0, float('inf')))
#m.y = Variable()

`x` and `y` are linopy variables of the class `linopy.Variable`. Each of them contain all relevant information that define it. The `name` parameter is optional but can be useful for referencing the variables later.

In [4]:
y

Variable
--------
y ∈ [0, inf]

## Adding the Objective 

The objective function defines what you want to optimize. You can set the objective function of a Model in Linopy using the add_objective method. For our example that would be

In [5]:
# Create Objective
m.add_objective(-4*x - 3*y)
#m.obj = Objective(4 * m.x + 3 * m.y, sense=Sense.MAXIMIZE)

LinearExpression
----------------
-4 x - 3 y

Since both `x` and `y` are scalar variables (meaning they don't have any dimensions), their underlying data contain only one optimization variable each.  

### Adding Constraints

Constraints define the feasible region of the optimization problem. They consist of the left hand side (lhs) and the right hand side (rhs). The first constraint that we want to write down is 
$x <= 4$, which we write out exactly in the mathematical way.

We assign it by calling the function `m.add_constraints`.

In [6]:
# Create Constraints

m.add_constraints(x  <= 4)
m.add_constraints(2*x + y <= 10)
m.add_constraints(x + y <= 10)




Constraint `con2`
-----------------
+1 x + 1 y ≤ 10

## Solving the Model

Once you've defined your Model with variables, constraints, and an objective function, you can solve it using the `solve` method:

In [7]:
# Solve the Problem
m.solve()
#m.solve(solver="cbc")  # You can choose a different solver like 'glpk', 'gurobi', etc.

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\huang\AppData\Local\Temp\linopy-problem-2zzuyndv.lp
Reading time = 0.01 seconds
obj: 3 rows, 2 columns, 5 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x36ba073a
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.0000000e+01   0.000000e+00   0.000000e+00      0s
       0   -3.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 ite

('ok', 'optimal')

In [20]:
x.solution.values

array(0.)

In [21]:
y.solution.values


array(10.)

In [10]:
# Print Solver Status
print("Solver Status:", m.status)

Solver Status: ok


While we can read from the message above that our problem was solved successfully, we can also formally check by accessing the reported status in the Linopy "Model()" object.

In [29]:
assert m.status == 'ok' 

### Dual values (aka shadow prices)

Access to dual values in scripts is similar to accessing primal variable values, except that dual values are not captured by default so additional directives are needed **before** optimization to signal that duals are desired.

To signal that duals are desired, declare a **Suffix** (another `pyomo` component) with the name "dual" on the model.

In [1]:
"""
# Get Dual Values
m.dual = m.get_duals()

# Print Dual Values
print("Dual for c1:", m.dual[m.c1])
print("Dual for c2:", m.dual[m.c2])

# Create a new Problem
m = Problem()

# Create Sets and Variables
m.A = range(1, 3)
B = ["wind", "solar"]
m.B = B
m.x = Variable(m.A)
m.y = Variable(m.A, B)

# Create Constraints
m.c1 = Constraint(sum(m.y[a, b] for b in B) == 1 for a in m.A)
m.c2 = Constraint(sum(m.y[a, b] for b in B) >= 1 for a in m.A)

# Solve the Problem
m.solve(solver="cbc")

# Print Variable Values
print("x:", [m.x[a].value for a in m.A])
print("y:")
for a in m.A:
    for b in B:
        print(f"y[{a}, {b}]:", m.y[a, b].value)

# Create a new Problem
m = Problem()

# Define Marginal Costs, Capacities, and Load
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
}
capacities = {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000}
load = 42000

# Create Sets and Variables
m.S = list(capacities.keys())
m.g = Variable(m.S, domain=(0, None))

# Create Objective
m.cost = Objective(sum(marginal_costs[s] * m.g[s] for s in m.S))

# Create Constraints
m.generator_limit = Constraint(m.g[s] <= capacities[s] for s in m.S)
m.energy_balance = Constraint(sum(m.g[s] for s in m.S) == load)

# Solve the Problem
m.solve(solver="cbc")

# Print Objective Value
print("Objective (Cost):", m.cost.value)

# Print Variable Values
print("Generators (g):")
for s in m.S:
    print(f"{s}: {m.g[s].value}")

# Get Dual Values
m.dual = m.get_duals()

# Print Dual Values
print("Dual for energy_balance:", m.dual[m.energy_balance])
for s in m.S:
    print(f"Dual for generator_limit ({s}): {m.dual[m.generator_limit[s]]}")

# Continue with the remaining code similarly.
"""

'\n# Get Dual Values\nm.dual = m.get_duals()\n\n# Print Dual Values\nprint("Dual for c1:", m.dual[m.c1])\nprint("Dual for c2:", m.dual[m.c2])\n\n# Create a new Problem\nm = Problem()\n\n# Create Sets and Variables\nm.A = range(1, 3)\nB = ["wind", "solar"]\nm.B = B\nm.x = Variable(m.A)\nm.y = Variable(m.A, B)\n\n# Create Constraints\nm.c1 = Constraint(sum(m.y[a, b] for b in B) == 1 for a in m.A)\nm.c2 = Constraint(sum(m.y[a, b] for b in B) >= 1 for a in m.A)\n\n# Solve the Problem\nm.solve(solver="cbc")\n\n# Print Variable Values\nprint("x:", [m.x[a].value for a in m.A])\nprint("y:")\nfor a in m.A:\n    for b in B:\n        print(f"y[{a}, {b}]:", m.y[a, b].value)\n\n# Create a new Problem\nm = Problem()\n\n# Define Marginal Costs, Capacities, and Load\nmarginal_costs = {\n    "Wind": 0,\n    "Coal": 30,\n    "Gas": 60,\n    "Oil": 80,\n}\ncapacities = {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000}\nload = 42000\n\n# Create Sets and Variables\nm.S = list(capacities.keys())\nm.g 